**Importing the Dependencies and getting the data from Kaggle**

In [1]:
! pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download -d wordsforthewise/lending-club

Dataset URL: https://www.kaggle.com/datasets/wordsforthewise/lending-club
License(s): CC0-1.0
100%|██████████████████████████████████████| 1.26G/1.26G [04:02<00:00, 7.35MB/s]
100%|██████████████████████████████████████| 1.26G/1.26G [04:02<00:00, 5.59MB/s]


In [8]:
! unzip lending-club.zip

Archive:  lending-club.zip
  inflating: accepted_2007_to_2018Q4.csv.gz  
  inflating: accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv  
  inflating: rejected_2007_to_2018Q4.csv.gz  
  inflating: rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv  


In [6]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

**Importing the Dataset**

We have to concatenate all CSV files

In [4]:
file_paths = ['/home/user/Desktop/Lending Club/Short Datasets/final_dataset_2.csv', '/home/user/Desktop/Lending Club/Short Datasets/final_dataset_1.csv', '/home/user/Desktop/Lending Club/Short Datasets/final_dataset_3.csv']

In [5]:
df_list = [pd.read_csv(file) for file in file_paths]
combined_df = pd.concat(df_list)
combined_df.to_csv('combined_file.csv', index=False)

This has given us the final dataset as combined_file.csv

Our combined dataset have more than 6 lac rows and over 100 columns

In [7]:
ds = pd.read_csv('/home/user/Desktop/Lending Club/combined_file.csv')

/tmp/ipykernel_23856/3512670734.py:1: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  ds = pd.read_csv('/home/user/Desktop/Lending Club/combined_file.csv')


In [7]:
ds.shape

(614640, 103)

Rows: 6,14,640  Columns: 103

In [4]:
ds.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,url
0,32169364,3975.0,3975.0,3975.0,36 months,24.50,157.00,F,F3,NaN,...,0.0,100.0,100.0,0.0,0.0,4400.0,3598.0,2400.0,0.0,NaN
1,75135982,20000.0,20000.0,20000.0,60 months,15.31,479.06,C,C5,Structural Engineer,...,3.0,92.0,28.6,0.0,0.0,366374.0,61346.0,41300.0,42396.0,NaN
2,19596300,8725.0,8725.0,8725.0,36 months,23.43,339.71,F,F1,bartender,...,1.0,100.0,80.0,0.0,0.0,110918.0,130744.0,6800.0,98118.0,NaN
3,71483062,5800.0,5800.0,5800.0,36 months,14.46,199.53,C,C4,Operations Manager,...,2.0,93.3,25.0,0.0,0.0,9000.0,1857.0,6000.0,0.0,NaN
4,71663088,25000.0,25000.0,25000.0,60 months,14.46,587.69,C,C4,Telecommunications Specialist,...,4.0,93.0,50.0,0.0,2.0,474659.0,79348.0,16950.0,67709.0,NaN


**Data Preprocessing**

In [8]:
# finding % of NULL values
null_percentage = ds.isnull().mean() * 100
print(null_percentage)

id                             0.000000
loan_amnt                      0.000000
funded_amnt                    0.000000
funded_amnt_inv                0.000000
term                           0.000000
                                ...    
tot_hi_cred_lim               46.109918
total_bal_ex_mort             42.816120
total_bc_limit                42.816120
total_il_high_credit_limit    46.109918
url                           59.325784
Length: 103, dtype: float64


In [8]:
# dropping the url column
ds.drop(columns=['url'], inplace=True)

In [10]:
pd.set_option('display.max_columns', None)
print(ds.head())

         id  loan_amnt  funded_amnt  funded_amnt_inv        term  int_rate  \
0  32169364     3975.0       3975.0           3975.0   36 months     24.50   
1  75135982    20000.0      20000.0          20000.0   60 months     15.31   
2  19596300     8725.0       8725.0           8725.0   36 months     23.43   
3  71483062     5800.0       5800.0           5800.0   36 months     14.46   
4  71663088    25000.0      25000.0          25000.0   60 months     14.46   

   installment grade sub_grade                      emp_title emp_length  \
0       157.00     F        F3                            NaN        NaN   
1       479.06     C        C5            Structural Engineer    8 years   
2       339.71     F        F1                      bartender    9 years   
3       199.53     C        C4             Operations Manager    2 years   
4       587.69     C        C4  Telecommunications Specialist    3 years   

  home_ownership  annual_inc verification_status   issue_d  loan_status  \

We have to now identify which factors will not affect the final output and should be dropped while training the model

In [11]:
ds.describe()

,id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,total_bal_il,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,6.146400e+05,614640.000000,614640.000000,614640.000000,614640.000000,614640.000000,6.146400e+05,614640.000000,614420.000000,614640.000000,614640.000000,614640.000000,614640.000000,307723.000000,614640.000000,614640.000000,6.146400e+05,614211.000000,614640.000000,614640.000000,614640.000000,614640.000000,614640.000000,614640.000000,614640.000000,6.146400e+05,614640.000000,614640.000000,614640.000000,614640.000000,614640.000000,614634.000000,614640.0,614640.000000,581230.000000,5.812300e+05,272716.000000,272716.000000,272716.000000,272716.000000,2.727160e+05,272716.000000,272716.000000,272716.000000,272691.000000,5.812300e+05,272716.000000,272716.000000,158731.000000,351475.000000,331220.00000,347486.000000,347250.000000,364634.000000,364640.000000,321002.000000,331229.000000,331229.000000,331230.000000,351475.000000,347790.000000,320731.000000,331230.000000,331230.000000,331230.000000,342915.000000,331230.000000,331230.000000,331230.000000,331230.000000,331230.000000,342915.000000,317923.000000,331230.000000,331230.000000,331230.000000,331208.000000,347421.000000,364522.000000,364639.000000,3.312300e+05,3.514750e+05,351475.000000,3.312300e+05
mean,5.927770e+07,14844.087393,14840.058652,14825.191288,14.222565,448.644822,7.426527e+04,0.528072,18.905330,0.329910,693.594934,697.595046,0.704614,34.075185,11.698132,0.223677,1.562541e+04,52.451761,24.899333,398.004197,397.914955,12596.593196,12586.447393,9561.824675,2503.400429,2.911527e+00,528.456564,88.054286,3946.161753,641.243090,610.450337,0.018494,1.0,0.005166,236.077706,1.338522e+05,1.097288,2.814188,0.817073,1.805772,3.608562e+04,1.490657,3.157398,5291.169242,59.050247,3.118535e+04,1.172568,1.599569,2.474161,4.810735,12733.44981,9168.491136,61.417321,0.008743,13.654514,124.811612,176.742302,12.447192,7.503934,1.578957,22.642221,6.536711,0.510262,3.684479,5.755330,4.718542,8.077451,8.595456,8.308142,14.549401,5.701914,11.629946,0.000793,0.003010,0.089494,2.244045,94.201029,47.136685,0.134634,0.049059,1.645200e+05,4.829130e+04,20111.971096,4.158210e+04
std,4.107960e+07,8840.893100,8838.664286,8837.882790,4.969873,264.267290,6.724129e+04,0.499212,12.267815,0.903122,30.478056,30.478615,0.970635,21.897596,5.545662,0.618498,2.050927e+04,24.448627,12.080286,2619.319065,2618.870061,9815.419870,9812.985913,8472.337512,2694.103839,1.484309e+01,1360.491493,239.536278,6459.446974,88.806587,167.490202,0.152162,0.0,0.079642,1887.559860,1.504643e+05,1.240982,3.027992,1.021600,1.728953,4.302062e+04,1.625423,2.799203,5277.674619,20.886935,3.130873e+04,1.657492,2.827096,2.668492,3.201063,15472.88146,14233.825736,28.089423,0.106633,739.776964,52.928728,94.038639,15.561346,8.316258,1.985792,29.344547,5.813800,1.324919,2.256715,3.312314,2.890215,4.762073,7.440089,4.516152,8.016895,3.238462,5.436689,

In [12]:
ds['loan_status'].value_counts()

loan_status
1    324574
0    290066
Name: count, dtype: int64

0 - Bad Loan    1 - Good Loan

**Label Encoding**

Columns where label encoding can be easily done:

grade, sub_grade, emp_title, home_ownership, verification_status, purpose, addr_state, initial_list_status, application_type

In [9]:
# Columns to be label encoded
columns_to_encode = ['grade', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'purpose', 'addr_state', 'initial_list_status', 'application_type']

# Initialize the LabelEncoder
label_encoders = {}

# Apply LabelEncoder to each column
for column in columns_to_encode:
    label_encoders[column] = LabelEncoder()
    ds[column] = label_encoders[column].fit_transform(ds[column])


Columns which probably need to be deleted going forward:

zip code, title

In [10]:
ds.drop(columns="zip_code", inplace = True)

In [11]:
ds.drop(columns=['title'], inplace = True)

Columns which are giving issues:

term, issue_d, earliest_cr_line, last_pymnt_d, last_credit_pull_d, payment_plan

In [12]:
ds.last_pymnt_d.head()

0    Oct-2015
1    Feb-2019
2    Aug-2015
3    Nov-2016
4    Jul-2017
Name: last_pymnt_d, dtype: object

In [13]:
ds[['issue_d_month', 'issue_d_year']] = ds['issue_d'].str.split('-', expand=True)

In [14]:
ds[['earliest_cr_line_month', 'earliest_cr_line_year']] = ds['earliest_cr_line'].str.split('-', expand=True)

In [15]:
ds[['last_payment_d_month', 'last_payment_d_year']] = ds['last_pymnt_d'].str.split('-', expand=True)

In [16]:
ds[['last_credit_pull_d_month', 'last_credit_pull_d_year']] = ds['last_credit_pull_d'].str.split('-', expand=True)

We have divided all the columns containing dates into 2 containing months and year:

issue_d, earliest_cr_line, last_payment_d, last_credit_pull_d

In [17]:
ds['pymnt_plan'].value_counts()

pymnt_plan
n    614037
y       603
Name: count, dtype: int64

In [18]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Color' column
ds['pymnt_plan'] = label_encoder.fit_transform(ds['pymnt_plan'])

# i was not writing the correct column name due to which it was showing me error in label encoding beforehand

In [19]:
ds['term'].value_counts()

term
36 months    432640
60 months    182000
Name: count, dtype: int64

In [20]:
# Extract numbers using a regular expression
ds['term_len'] = ds['term'].str.extract('(\d+)')

# Convert the extracted numbers to integers
ds['term_len'] = ds['term_len'].astype(int)


We have to look into the distribution of emp_length function to give appropriate value to NULL values

In [21]:
ds.emp_length.head()

0        NaN
1    8 years
2    9 years
3    2 years
4    3 years
Name: emp_length, dtype: object

In [22]:
# Columns to delete
columns_to_delete = ['term', 'emp_length']

# Delete the specified columns
ds = ds.drop(columns=columns_to_delete)

We need to delete the date columns which have already been divided

In [23]:
# Columns to delete
columns_to_delete = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']

# Delete the specified columns
ds = ds.drop(columns=columns_to_delete)

In [24]:
pd.set_option('display.max_columns', None)
print(ds.head())

         id  loan_amnt  funded_amnt  funded_amnt_inv  int_rate  installment  \
0  32169364     3975.0       3975.0           3975.0     24.50       157.00   
1  75135982    20000.0      20000.0          20000.0     15.31       479.06   
2  19596300     8725.0       8725.0           8725.0     23.43       339.71   
3  71483062     5800.0       5800.0           5800.0     14.46       199.53   
4  71663088    25000.0      25000.0          25000.0     14.46       587.69   

   grade  sub_grade  emp_title  home_ownership  annual_inc  \
0      5         27     191371               4     13622.0   
1      2         14     144225               1     89460.0   
2      5         25     166200               5     26000.0   
3      2         13     100879               5     38000.0   
4      2         13     150407               1     97000.0   

   verification_status  loan_status  pymnt_plan  purpose  addr_state    dti  \
0                    2            0           0        0           3  13.

Now we need to label encode the divided dates columns:

issue_d_month, earliest_cr_line_month, last_payment_d_month, last_credit_pull_d_month

In [25]:
# Columns to be label encoded
columns_to_encode = ['issue_d_month', 'earliest_cr_line_month', 'last_payment_d_month', 'last_credit_pull_d_month']

# Initialize the LabelEncoder
label_encoders = {}

# Apply LabelEncoder to each column
for column in columns_to_encode:
    label_encoders[column] = LabelEncoder()
    ds[column] = label_encoders[column].fit_transform(ds[column])

Label Encoding is Complete

**Handling Missing Data**

We have to first check how many missing values are present in different columns and accordingly decide what to do with it

In [26]:
# finding % of NULL values
pd.set_option('display.max_rows', None)
null_percentage = ds.isnull().mean() * 100
print(null_percentage)

id                             0.000000
loan_amnt                      0.000000
funded_amnt                    0.000000
funded_amnt_inv                0.000000
int_rate                       0.000000
installment                    0.000000
grade                          0.000000
sub_grade                      0.000000
emp_title                      0.000000
home_ownership                 0.000000
annual_inc                     0.000000
verification_status            0.000000
loan_status                    0.000000
pymnt_plan                     0.000000
purpose                        0.000000
addr_state                     0.000000
dti                            0.035793
delinq_2yrs                    0.000000
fico_range_low                 0.000000
fico_range_high                0.000000
inq_last_6mths                 0.000000
mths_since_last_delinq        49.934433
open_acc                       0.000000
pub_rec                        0.000000
revol_bal                      0.000000


In [28]:
ds.shape

(614640, 103)

Filling up the NULL values

In [29]:
# using mode

# Calculate the mode for each column
modes = ds.mode().iloc[0]

# Fill null values with the mode of each column
ds = ds.fillna(modes)

In [25]:
ds.shape

(614640, 103)

In [30]:
# finding % of NULL values
pd.set_option('display.max_rows', None)
null_percentage = ds.isnull().mean() * 100
print(null_percentage)

id                            0.0
loan_amnt                     0.0
funded_amnt                   0.0
funded_amnt_inv               0.0
int_rate                      0.0
installment                   0.0
grade                         0.0
sub_grade                     0.0
emp_title                     0.0
home_ownership                0.0
annual_inc                    0.0
verification_status           0.0
loan_status                   0.0
pymnt_plan                    0.0
purpose                       0.0
addr_state                    0.0
dti                           0.0
delinq_2yrs                   0.0
fico_range_low                0.0
fico_range_high               0.0
inq_last_6mths                0.0
mths_since_last_delinq        0.0
open_acc                      0.0
pub_rec                       0.0
revol_bal                     0.0
revol_util                    0.0
total_acc                     0.0
initial_list_status           0.0
out_prncp                     0.0
out_prncp_inv 

Exporting to a csv file just to save the changes made till now

In [26]:
# exporting as another csv file
ds.to_csv('non_null_output.csv', index=False)

**Separating Data and Labels**

In [31]:
X = ds.drop(columns = 'loan_status', axis = 1)
Y = ds['loan_status']   

**Training and Testing Data**

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=1)

In [35]:
print(X.shape, X_train.shape, X_test.shape)

(614640, 102) (491712, 102) (122928, 102)


**Standardizing the Data**

In [36]:
scaler = StandardScaler()

In [37]:
scaler.fit(X_train)

StandardScaler()

**Model Training - Logistic Regression**

In [38]:
model = LogisticRegression()

In [39]:
# training the logistic regression model with training data
model.fit(X_train, Y_train)

/home/user/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

**Model Evaluation**

Accuracy on Training Data

In [40]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train) 

In [42]:
print('Accuracy on training data : ', training_data_accuracy)

Accuracy on training data :  0.8540019360926722


Accuracy on Test Data

In [43]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test) 

In [44]:
print('Accuracy on test data : ', test_data_accuracy)

Accuracy on test data :  0.8530277886242353
